In [2]:
import pandas as pd
import numpy as np

In [15]:
diagnosis_sample = pd.read_csv('./DiagnosisSample.csv')
diagnosis_sample['DiagnosisDate'] = pd.to_datetime(diagnosis_sample['DiagnosisDate'])
# diagnosis_sample = diagnosis_sample.loc[diagnosis_sample.IsCancerDiagnosis]
diagnosis_sample.head()

,PatientID,DiagnosisDate,DiagnosisCode,Diagnosis,IsCancerDiagnosis
0,2038,2010-01-21,174.9,Breast Cancer,True
1,2120,2010-01-09,174.1,Breast Cancer,True
2,2175,2010-02-17,174.7,Breast Cancer,True
3,2407,2010-06-13,174.9,Breast Cancer,True
4,2425,2010-12-15,174.9,Breast Cancer,True


In [7]:
treatment_sample = pd.read_csv('./TreatmentSample.csv')
treatment_sample['TreatmentDate'] = pd.to_datetime(treatment_sample['TreatmentDate'])
treatment_sample.head()

,PatientID,TreatmentDate,DrugCode
0,2038,2010-01-24,A
1,2120,2010-01-24,A
2,2120,2010-01-24,B
3,2120,2010-01-26,A
4,2120,2010-01-26,B


In [33]:
# Q1
colon_caner_patient = diagnosis_sample.loc[diagnosis_sample.Diagnosis == 'Colon Cancer'].sort_values(by=['PatientID', 'DiagnosisDate'])
# colon_caner_patient = colon_caner_patient[['PatientID', 'DiagnosisDate']].drop_duplicates(keep='first')
# 去问面试官 取最大时间还是最小时间
colon_caner_patient = colon_caner_patient[['PatientID', 'DiagnosisDate']].groupby(['PatientID']).min().reset_index() 
colon_caner_patient = colon_caner_patient.merge(treatment_sample, on='PatientID', how='left')
# filter treatment 比 发现时间早, 这里要不要筛 也可以问一下面试官
colon_caner_patient = colon_caner_patient.loc[colon_caner_patient.TreatmentDate >= colon_caner_patient.DiagnosisDate]
colon_caner_patient = colon_caner_patient[['PatientID', 'DiagnosisDate', 'TreatmentDate']].groupby(['PatientID', 'DiagnosisDate']).min().reset_index()
colon_caner_patient['gap'] = (colon_caner_patient.TreatmentDate - colon_caner_patient.DiagnosisDate).dt.days
colon_caner_patient.gap.mean()

5.555555555555555

In [58]:
# Q2
cancer_patient = diagnosis_sample.loc[diagnosis_sample.IsCancerDiagnosis]
cancer_patient = cancer_patient.merge(treatment_sample, on=['PatientID'])
# 拿一线用药，去看第一天用的所有药
# 去算每个人的第一天，取出第一天用了啥
patient_first_day = cancer_patient[['PatientID', 'TreatmentDate']].groupby(['PatientID']).min().reset_index()
patient_first_day_drug = cancer_patient.merge(patient_first_day, on=['PatientID', 'TreatmentDate'])
patient_first_day_drug = patient_first_day_drug[['PatientID', 'DrugCode']].drop_duplicates()

# 每个人治疗过程中用了哪些药
all_time_drug = cancer_patient[['PatientID', 'DrugCode']].drop_duplicates()
second_drug = all_time_drug.merge(patient_first_day_drug, on=['PatientID', 'DrugCode'], how='left', indicator=True)
second_drug = second_drug.loc[second_drug._merge == 'left_only']
ttl_patient = all_time_drug.PatientID.nunique()
second_patient = second_drug.PatientID.nunique()
print(f"in ttl patient {ttl_patient}, {second_patient}({second_patient/ttl_patient * 100:.2f}%) use second")

in ttl patient 28, 9(32.14%) use second


In [42]:
# q3
df = diagnosis_sample.merge(treatment_sample, on='PatientID')
df = df.loc[df.DrugCode == 'C']
first_treat_time = df[['PatientID', 'TreatmentDate']].groupby(['PatientID']).min().reset_index().rename(columns={'TreatmentDate': 'first_date'})
last_treat_time = df[['PatientID', 'TreatmentDate']].groupby(['PatientID']).max().reset_index().rename(columns={'TreatmentDate': 'last_date'})
drug_c = first_treat_time.merge(last_treat_time, on=['PatientID'])
drug_c['gap'] = (drug_c['last_date'] - drug_c['first_date']).dt.days
drug_c['gap'].mean()

110.47058823529412